## Preprocessing

In [211]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df_master = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df_master.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [212]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df_master.drop(columns=['EIN', 'NAME'])

In [213]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [214]:
# Look at APPLICATION_TYPE value counts for binning
type_counts = application_df['APPLICATION_TYPE'].value_counts()
type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [215]:
income_values = application_df['INCOME_AMT'].value_counts()
income_values

INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [216]:
copy_df = application_df.copy()

copy_df['INCOME_AMT'].replace('1-9999', '1', inplace=True)
copy_df['INCOME_AMT'].replace('10000-24999', '1', inplace=True)
copy_df['INCOME_AMT'].replace('25000-99999', '2', inplace=True)
copy_df['INCOME_AMT'].replace('100000-499999', '3', inplace=True)
copy_df['INCOME_AMT'].replace('1M-5M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('5M-10M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('10M-50M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('50M+', '4', inplace=True)
copy_df = copy_df.astype({'INCOME_AMT': 'int64'})


In [217]:
copy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  int64 
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 2.6+ MB


In [218]:

type_counts.iloc[7:]


APPLICATION_TYPE
T10    528
T9     156
T13     66
T12     27
T2      16
T25      3
T14      3
T29      2
T15      2
T17      1
Name: count, dtype: int64

In [219]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts.iloc[7:].index)

# Replace in dataframe
for app in application_types_to_replace:
    copy_df['APPLICATION_TYPE'] = copy_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
copy_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [220]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()

In [221]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_clean = class_counts.loc[class_counts > 1]
class_clean

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [222]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts.loc[class_counts < 1883].index)

# Replace in dataframe
for cls in classifications_to_replace:
    copy_df['CLASSIFICATION'] = copy_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
copy_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [223]:
copy_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,1,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,3,N,142590,1


In [224]:
copy_df = copy_df.drop(columns=['STATUS', 'SPECIAL_CONSIDERATIONS'], axis=0)
#copy_df = application_df.copy()

In [225]:
copy_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,6692,1
4,T3,Independent,C1000,Heathcare,Trust,3,142590,1


In [226]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_data = copy_df.dtypes[copy_df.dtypes == "object"].index.tolist()
cat_df = pd.DataFrame(copy_df[columns:= cat_data])
concat_df = pd.get_dummies(cat_df, dtype= int)


In [227]:
trial_1_df = copy_df.drop(columns= cat_data, axis=0)

In [228]:
copy_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,6692,1
4,T3,Independent,C1000,Heathcare,Trust,3,142590,1


In [229]:
trial_1_df = pd.concat([trial_1_df, concat_df], axis=1)


In [230]:
trial_1_df.head()


,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,0,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,0,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,3,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [231]:
# Split our preprocessed data into our features and target arrays
y = trial_1_df['IS_SUCCESSFUL'].values
X = trial_1_df.drop(columns= ['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [232]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(

## Compile, Train and Evaluate the Model

In [233]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 9
hidden_nodes_layer2 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,
                             activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_45 (Dense)                │ (None, 9)              │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 7)              │            70 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │             8 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 366 (1.43 KB)

 Trainable params: 366 (1.43 KB)

 Non-trainable params: 0 (0.00 B)

In [234]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=20)

Epoch 1/20


804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 954us/step - accuracy: 0.6988 - loss: 0.6099
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 942us/step - accuracy: 0.7225 - loss: 0.5654
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 937us/step - accuracy: 0.7289 - loss: 0.5566
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 949us/step - accuracy: 0.7317 - loss: 0.5520
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 915us/step - accuracy: 0.7308 - loss: 0.5531
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 918us/step - accuracy: 0.7291 - loss: 0.5590
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.7260 - loss: 0.5571
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 907us/step - accuracy: 0.7270 - loss: 0.5555
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 966us/step - accuracy: 0.7242 - loss: 0.5566
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 917us/step - accuracy: 0.7324 - loss: 0.5530
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 970us/step - accuracy: 0.7334 - loss: 0.5513
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 

In [235]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7314 - loss: 0.5521
Loss: 0.5521377921104431, Accuracy: 0.7314285635948181


In [236]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [237]:
copy_df = application_df.copy()

copy_df['INCOME_AMT'].replace('1-9999', '1', inplace=True)
copy_df['INCOME_AMT'].replace('10000-24999', '1', inplace=True)
copy_df['INCOME_AMT'].replace('25000-99999', '2', inplace=True)
copy_df['INCOME_AMT'].replace('100000-499999', '3', inplace=True)
copy_df['INCOME_AMT'].replace('1M-5M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('5M-10M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('10M-50M', '4', inplace=True)
copy_df['INCOME_AMT'].replace('50M+', '4', inplace=True)
copy_df = copy_df.astype({'INCOME_AMT': 'int64'})
copy_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,1,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,3,N,142590,1


In [238]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts.iloc[7:].index)

# Replace in dataframe
for app in application_types_to_replace:
    copy_df['APPLICATION_TYPE'] = copy_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
copy_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [239]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_data = copy_df.dtypes[copy_df.dtypes == "object"].index.tolist()
cat_df = pd.DataFrame(copy_df[columns:= cat_data])
concat_df = pd.get_dummies(cat_df, dtype= int)


In [240]:
trial_2_df = copy_df.drop(columns= cat_data, axis=0)

In [241]:
trial_2_df = pd.concat([trial_2_df, concat_df], axis=1)


In [242]:
# Split our preprocessed data into our features and target arrays
y = trial_2_df['IS_SUCCESSFUL'].values
X = trial_2_df.drop(columns= ['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing datset
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, random_state=42)

In [243]:
# Fit the StandardScaler
X_scaler_2 = scaler.fit(X_train_2)

# Scale the data
X_train_scaled_2 = X_scaler.transform(X_train_2)
X_test_scaled_2 = X_scaler.transform(X_test_2)

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(

In [244]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled_2[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 10

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,
                             activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_48 (Dense)                │ (None, 20)             │         2,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,221 (8.68 KB)

 Trainable params: 2,221 (8.68 KB)

 Non-trainable params: 0 (0.00 B)

In [245]:
# Train the model
fit_model_2 = nn2.fit(X_train_scaled_2, y_train_2, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6960 - loss: 0.6085
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7280 - loss: 0.5543
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7291 - loss: 0.5530
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7212 - loss: 0.5585
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7319 - loss: 0.5481
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7298 - loss: 0.5491
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7362 - loss: 0.5445
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7316 - loss: 0.5485
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7367 - loss: 0.5438
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7246 - loss: 0.5506
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7347 - loss: 0.5456
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [246]:
# Evaluate the model using the test data
model_loss_2, model_accuracy_2 = nn2.evaluate(X_test_scaled_2, y_test_2,verbose=2)
print(f"Loss: {model_loss_2}, Accuracy: {model_accuracy_2}")

268/268 - 0s - 1ms/step - accuracy: 0.7315 - loss: 0.5535
Loss: 0.5534582138061523, Accuracy: 0.7315452098846436


In [247]:
application_df['INCOME_AMT'].value_counts()


INCOME_AMT
0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: count, dtype: int64

In [248]:
copy_df = application_df.copy()

copy_df['INCOME_AMT'].replace('1-9999', '5000', inplace=True)
copy_df['INCOME_AMT'].replace('10000-24999', '17500', inplace=True)
copy_df['INCOME_AMT'].replace('25000-99999', '50000', inplace=True)
copy_df['INCOME_AMT'].replace('100000-499999', '250000', inplace=True)
copy_df['INCOME_AMT'].replace('1M-5M', '3000000', inplace=True)
copy_df['INCOME_AMT'].replace('5M-10M', '7500000', inplace=True)
copy_df['INCOME_AMT'].replace('10M-50M', '30000000', inplace=True)
copy_df['INCOME_AMT'].replace('50M+', '50000000', inplace=True)
copy_df = copy_df.astype({'INCOME_AMT': 'int64'})
copy_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,5000,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,17500,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,250000,N,142590,1


In [249]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(type_counts.iloc[7:].index)

# Replace in dataframe
for app in application_types_to_replace:
    copy_df['APPLICATION_TYPE'] = copy_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
copy_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: count, dtype: int64

In [250]:
# Convert categorical data to numeric with `pd.get_dummies`
cat_data = copy_df.dtypes[copy_df.dtypes == "object"].index.tolist()
cat_df = pd.DataFrame(copy_df[columns:= cat_data])
concat_df = pd.get_dummies(cat_df, dtype= int)


In [251]:
trial_3_df = copy_df.drop(columns= cat_data, axis=0)
trial_3_df = pd.concat([trial_2_df, concat_df], axis=1)


In [252]:
# Split our preprocessed data into our features and target arrays
y = trial_3_df['IS_SUCCESSFUL'].values
X = trial_3_df.drop(columns= ['IS_SUCCESSFUL'], axis=1)

# Split the preprocessed data into a training and testing datset
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, random_state=42)

In [253]:
# Fit the StandardScaler
X_scaler_3 = scaler.fit(X_train_3)

# Scale the data
X_train_scaled_3 = X_scaler.transform(X_train_3)
X_test_scaled_3 = X_scaler.transform(X_test_3)

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(

In [254]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled_3[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 5

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features,
                             activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

c:\Users\pkare\anaconda3\envs\dev\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_51 (Dense)                │ (None, 20)             │         3,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,191 (16.37 KB)

 Trainable params: 4,191 (16.37 KB)

 Non-trainable params: 0 (0.00 B)

In [255]:
# Train the model
fit_model_3 = nn3.fit(X_train_scaled_3, y_train_3, epochs=20)

Epoch 1/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.6814 - loss: 0.6376
Epoch 2/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7183 - loss: 0.5722
Epoch 3/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7308 - loss: 0.5511
Epoch 4/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7298 - loss: 0.5514
Epoch 5/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7278 - loss: 0.5551
Epoch 6/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7261 - loss: 0.5545
Epoch 7/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7321 - loss: 0.5479
Epoch 8/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7296 - loss: 0.5506
Epoch 9/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7349 - loss: 0.5475
Epoch 10/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7351 - loss: 0.5457
Epoch 11/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7326 - loss: 0.5441
Epoch 12/20
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

In [256]:
# Evaluate the model using the test data
model_loss_3, model_accuracy_3 = nn3.evaluate(X_test_scaled_3, y_test_3,verbose=2)
print(f"Loss: {model_loss_3}, Accuracy: {model_accuracy_3}")

268/268 - 0s - 2ms/step - accuracy: 0.7301 - loss: 0.5495
Loss: 0.549481213092804, Accuracy: 0.7301457524299622


In [ ]:
# Export our model to HDF5 file
from tensorflow.keras.models import save_model
save_model(nn, "../models/model_optimized.h5")